In [1]:
import _pickle as cPickle
import city_simulation
import gym
import random
import torch
from umap import UMAP
from sklearn.decomposition import PCA

pca_2 = cPickle.load(open('../sumo_simulation/input/pca_v2.pkl','rb'))

In [2]:
import numpy as np

In [8]:
import os, sys

if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

import traci

In [4]:
sumoBinary = "/usr/bin/sumo/bin/sumo"

In [5]:
path_list = [
    "../sumo_simulation/sim_config/km2_centro/scenario/osm.sumocfg",
    "../sumo_simulation/sim_config/km2_centro_2/scenario/osm.sumocfg",
    "../sumo_simulation/sim_config/km2_jesusmaria/scenario/osm.sumocfg",
    "../sumo_simulation/sim_config/km2_lince/scenario/osm.sumocfg",
    "../sumo_simulation/sim_config/km2_miraflores/scenario/osm.sumocfg",
    "../sumo_simulation/sim_config/km2_sani/scenario/osm.sumocfg"
]

In [9]:
def run_day_sim(path):
    sumoCmd = [sumoBinary, "-c", path]
    traci.start(sumoCmd)

    for edge_id in traci.lane.getIDList():
        traci.lane.subscribe(edge_id, [traci.constants.VAR_CO2EMISSION,
                                       traci.constants.VAR_COEMISSION,
                                       traci.constants.VAR_PMXEMISSION,
                                       traci.constants.VAR_NOXEMISSION,
                                       traci.constants.VAR_NOISEEMISSION,
                                       traci.constants.VAR_FUELCONSUMPTION])
    
    traci.simulationStep(500)    
    subscriptions = traci.lane.getAllSubscriptionResults()
    traci.close(False)
    return subscriptions

In [12]:
states = [run_day_sim(path) for path in path_list]

 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds


In [13]:
import pandas as pd

In [14]:
states = [pd.DataFrame.from_dict(state) for state in states]

In [15]:
us = [UMAP(n_components=2).fit(state) for state in states]

/home/andres/anaconda3/envs/pytorch/lib/python3.7/site-packages/umap/umap_.py:1385: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  "n_neighbors is larger than the dataset size; truncating to "
/home/andres/anaconda3/envs/pytorch/lib/python3.7/site-packages/umap/umap_.py:349: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "fuzzy_simplicial_set" failed type inference due to: Untyped global name 'nearest_neighbors': cannot determine Numba type of <class 'function'>

File "../../../anaconda3/envs/pytorch/lib/python3.7/site-packages/umap/umap_.py", line 467:
def fuzzy_simplicial_set(
    <source elided>
    if knn_indices is None or knn_dists is None:
        knn_indices, knn_dists, _ = nearest_neighbors(
        ^

  @numba.jit()
/home/andres/anaconda3/envs/pytorch/lib/python3.7/site-packages/numba/compiler.py:725: NumbaWarning: Function "fuzzy_simplicial_set" was compiled in object mode without 

In [18]:
import _pickle as cPickle

In [19]:
zones = ['km2_centro',
        'km2_centro_2',
        'km2_jesusmaria',
        'km2_lince',
        'km2_miraflores',
        'km2_sani']

for i, u in enumerate(us):
    cPickle.dump(u, open('../sumo_simulation/input/umap_{}.pkl'.format(zones[i]), 'wb'))